In [ ]:

from dotenv import load_dotenv
import os
import sys
import json
import pandas as pd
import time
import datetime
load_dotenv()

path = os.path.abspath('')
module_path = os.path.join(path,'..\..')
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"\\functions")


from dataset_utils import DatasetEvaluator

In [2]:
def save_queries(queries, FILE_NAME_PATH):
    data = {"queries":queries}
    with open(FILE_NAME_PATH, "w") as arquivo_json:
        json.dump(data, arquivo_json, indent=4) 

def read_queries(FILE_NAME_PATH):
    with open(FILE_NAME_PATH, encoding="utf-8", errors="ignore") as json_data:
        data = json.load(json_data, strict=False)
    queries = data["queries"]
    return queries

def get_schema(FILE_NAME_PATH):
    if "mondial_gpt" in FILE_NAME_PATH:
        return "mondial_gpt"
    elif "mondial" in FILE_NAME_PATH:
        return "mondial"
    return ""

def get_dataset_evaluator(FILE_NAME_PATH):
    SCHEMA = get_schema(FILE_NAME_PATH)  
    PREFIX = SCHEMA  
    if SCHEMA == "mondial_gpt":
        PREFIX = "mondial"
    print(PREFIX)
    print(SCHEMA)
    dataset_eval = DatasetEvaluator(dataset_file_path=f"../../datasets/{PREFIX}/{PREFIX}_dataset.json",
                                dataset_tables_path=f"../../datasets/{PREFIX}/result_tables/",
                                db_connection_file=f"../../datasets/{SCHEMA}_db_connection.json",
                                dataset_name=SCHEMA)
    return dataset_eval

In [3]:
path = "results/sql_coder_mondial.json"

In [4]:
dval = get_dataset_evaluator(path)

mondial
mondial


In [5]:
dval.dataset

[{'id': '1',
  'type': 'simple',
  'number_of_results': '1',
  'number_of_joins': '0',
  'number_of_filters': '0',
  'number_of_aggregations': '0',
  'question': 'What is the area of Thailand?',
  'query': "SELECT max(area) FROM mondial_country where name  = 'Thailand'",
  'keywords': '',
  'answer': [{'uri': '/results_table/1.csv'}]},
 {'id': '2',
  'type': 'simple',
  'number_of_results': '1056',
  'number_of_joins': '0',
  'number_of_filters': '1',
  'number_of_aggregations': '0',
  'question': 'What are the provinces with an area greater than 10000?',
  'query': 'SELECT name FROM mondial_province where area > 10000',
  'keywords': '',
  'answer': [{'uri': '/results_table/2.csv'}]},
 {'id': '3',
  'type': 'medium',
  'number_of_results': '1',
  'number_of_joins': '1',
  'number_of_filters': '1',
  'number_of_aggregations': '0',
  'question': 'What are the languages spoken in Poland?',
  'query': "SELECT mondial_language.name FROM mondial_language INNER JOIN mondial_country ON mondia

In [6]:
queries = read_queries(path)
queries

[{'question': 'What is the area of Thailand?',
  'tables': ['mondial_country'],
  'ddl': 'CREATE TABLE "MONDIAL_GPT"."MONDIAL_COUNTRY" \n   (\t"NAME" VARCHAR2(50 BYTE), \n\t"CODE" VARCHAR2(4 BYTE), \n\t"CAPITAL" VARCHAR2(50 BYTE), \n\t"PROVINCE" VARCHAR2(50 BYTE), \n\t"AREA" NUMBER, \n\t"POPULATION" NUMBER, \n\t"META_REPCOL" VARCHAR2(4000 BYTE)\n   ) SEGMENT CREATION IMMEDIATE \n  PCTFREE 10 PCTUSED 40 INITRANS 1 MAXTRANS 255 \n NOCOMPRESS LOGGING\n  STORAGE(INITIAL 65536 NEXT 1048576 MINEXTENTS 1 MAXEXTENTS 2147483645\n  PCTINCREASE 0 FREELISTS 1 FREELIST GROUPS 1\n  BUFFER_POOL DEFAULT FLASH_CACHE DEFAULT CELL_FLASH_CACHE DEFAULT)\n  TABLESPACE "MONDIAL_GPT" ;',
  'ddl_postgres': 'CREATE TABLE "MONDIAL_GPT"."MONDIAL_COUNTRY" \n   ("NAME" VARCHAR(50), \n\t"CODE" VARCHAR(4), \n\t"CAPITAL" VARCHAR(50), \n\t"PROVINCE" VARCHAR(50), \n\t"AREA" NUMERIC, \n\t"POPULATION" NUMERIC, \n\t"META_REPCOL" VARCHAR(4000)\n   );',
  'sql_coder': "\nSELECT MONDIAL_COUNTRY.AREA\nFROM MONDIAL_COUNTRY\nWHE

In [7]:
i = 0
new_queries = []
for q in queries:
    data =  dval.dataset[i]
    nq = {"id" : data["id"], "question": data["question"], "query_string": q["sql_coder_oracle"], "type": data["type"]}
    new_queries.append(nq)   
    i+=1

In [8]:
new_queries

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': "SELECT MONDIAL_COUNTRY.AREA\nFROM MONDIAL_COUNTRY\nWHERE UPPER(MONDIAL_COUNTRY.NAME) LIKE UPPER('%Thailand%')",
  'type': 'simple'},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': 'SELECT SUM(AREA) AS total_area\nFROM MONDIAL_GPT.MONDIAL_PROVINCE\nWHERE AREA > 10000',
  'type': 'simple'},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': "SELECT name\nFROM mondial_gpt.mondial_country\nWHERE UPPER(name) LIKE '%POLAND%'",
  'type': 'medium'},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': 'SELECT "DEPTH"\nFROM "MONDIAL_GPT"."MONDIAL_LAKE"\nWHERE LOWER("NAME") LIKE \'%lake%kariba%\'',
  'type': 'simple'},
 {'id': '5',
  'question': 'What is the total of provinces of Netherlands?',
  'query_string': "SELECT COUNT(*) AS total_provinces\nFROM MONDIAL_GPT.MONDIAL_PROVINCE\nWHERE COUNTRY = 'NL';"

In [9]:
save_queries(new_queries, "results/1_sql_coder_mondial_gpt.json")